In [ ]:
# 📌 Cell 1: Fetch markdown from GitHub and inject Agentic AI intro section

import requests

# ✅ Correct raw GitHub markdown URL
url = "https://raw.githubusercontent.com/PrabuAppDev/github-pages-genai/main/crew-ai-multi-agents.md"

# Fetch content
response = requests.get(url)
original_md = response.text

# Agentic AI introduction paragraph
agentic_intro = (
    "## Introduction to Agentic AI\n\n"
    "Agentic AI is the next evolution beyond Retrieval-Augmented Generation (RAG). "
    "Rather than relying on a single model to answer questions from documents, "
    "Agentic AI orchestrates multiple specialized LLM agents, each with a defined role. "
    "These agents operate in a response → action loop, interacting with tools like OCR engines, databases, or APIs "
    "to complete complex workflows autonomously.\n\n"
)

# Insert intro after front matter (--- blocks)
parts = original_md.split('---')
if len(parts) >= 3:
    parts[2] = "\n" + agentic_intro + parts[2]
    updated_md = '---'.join(parts)
else:
    updated_md = agentic_intro + original_md  # Fallback if no front matter

# Show preview
print("\n".join(updated_md.strip().splitlines()[:20]))  # Show first 20 lines

In [ ]:
# 📌 Cell 2: Strip markdown formatting to prepare plain narration text

import re
from bs4 import BeautifulSoup

def markdown_to_narration_text(md_text):
    # Remove front matter if still present
    if md_text.startswith('---'):
        md_text = re.split(r'^---$', md_text, flags=re.MULTILINE)[-1]

    # Remove Markdown links but keep link text
    md_text = re.sub(r'\[(.*?)\]\((.*?)\)', r'\1', md_text)

    # Remove HTML tags (e.g., <a href=...>)
    md_text = BeautifulSoup(md_text, "html.parser").get_text()

    # Remove bold/italic markers
    md_text = re.sub(r'[_*`#~]', '', md_text)

    # Replace bullet points and headings with newlines
    md_text = re.sub(r'^\s*[-*+]\s+', '\n• ', md_text, flags=re.MULTILINE)
    md_text = re.sub(r'^#{1,6}\s*', '\n', md_text, flags=re.MULTILINE)

    # Collapse multiple newlines
    md_text = re.sub(r'\n{2,}', '\n\n', md_text)

    return md_text.strip()

# Clean narration text
narration_text = markdown_to_narration_text(updated_md)

# Show a short preview
print("\n".join(narration_text.splitlines()[:20]))

In [ ]:
# 📌 Cell 3: Convert voiceover intro text to male audio using OpenAI TTS (voice: echo)

import os
import openai

# 🔐 Ensure your API key is loaded securely via environment variable
openai.api_key = os.getenv("OPENAI_API_KEY")

# Voiceover script
intro_text = (
    "Hi, I’m the AI voice of Prabu Arumugam. Trusted technology advisor with experience in systems architecture and software engineering. "
    "Specialized in large-scale systems transformations for the property and casualty insurance industry. "
    "I’m a pragmatic problem-solver with a proven track record of delivering results. "
    "With over 20 years of experience in P&C core systems and digital transformation — covering data migration, "
    "underwriting, pricing, rating, and policy administration — I bring expertise in solution architecture and systems modernization "
    "for P&C core applications and digital innovation."
)

# Generate and save MP3
speech_response = openai.audio.speech.create(
    model="tts-1",
    voice="echo",  # Male voice
    input=intro_text
)

# Save audio file
with open("intro_prabu.mp3", "wb") as f:
    f.write(speech_response.content)

print("✅ Voiceover saved as 'intro_prabu.mp3'")

In [ ]:
# 📌 Cell 4: Embed the generated voiceover MP3 using an HTML audio tag in the markdown

# Define the audio player HTML
audio_embed_html = (
    '\n<audio controls>\n'
    '  <source src="intro_prabu.mp3" type="audio/mpeg">\n'
    '  Your browser does not support the audio element.\n'
    '</audio>\n\n'
)

# Inject after the Agentic AI intro (we'll use its heading to locate insert point)
inject_point = "## Introduction to Agentic AI"
if inject_point in updated_md:
    updated_md_with_audio = updated_md.replace(inject_point, f"{inject_point}\n\n{audio_embed_html}", 1)
else:
    updated_md_with_audio = audio_embed_html + updated_md  # Fallback if not found

# Save to new markdown file
output_md_path = "crew-ai-with-audio.md"
with open(output_md_path, "w", encoding="utf-8") as f:
    f.write(updated_md_with_audio)

print(f"✅ Markdown with embedded audio saved as '{output_md_path}'")

In [ ]:
# 📌 Cell 5: Split narration text into sections for TTS processing

import textwrap

# Use the cleaned narration text from Cell 2
lines = narration_text.splitlines()

# Group lines into sections based on H2/H3-style headings (e.g., Crew AI, Agents, Workflow, etc.)
sections = []
current_section = []

for line in lines:
    if line.strip().startswith("##") or line.strip().startswith("###"):
        if current_section:
            sections.append("\n".join(current_section).strip())
            current_section = []
    current_section.append(line)

# Append the final section
if current_section:
    sections.append("\n".join(current_section).strip())

# Wrap paragraphs for easier debugging/printing
for idx, section in enumerate(sections):
    print(f"\n🧩 Section {idx + 1}\n" + "-"*20)
    print(textwrap.shorten(section.replace("\n", " "), width=300, placeholder="..."))

# Show number of sections
print(f"\n✅ Total sections prepared for TTS: {len(sections)}")

In [ ]:
# 📌 Cell 7: Embed Section 1 MP3 audio player after its heading in the markdown

# Define HTML audio block
section_audio_html = (
    '\n<audio controls>\n'
    '  <source src="section_1.mp3" type="audio/mpeg">\n'
    '  Your browser does not support the audio element.\n'
    '</audio>\n\n'
)

# Find the insertion point again
injection_heading = "## Introduction to Agentic AI"

# Inject the player just after the heading
if injection_heading in updated_md_with_audio:
    updated_md_with_section_audio = updated_md_with_audio.replace(
        injection_heading,
        f"{injection_heading}\n\n{section_audio_html}",
        1
    )
else:
    # Fallback if heading not found
    updated_md_with_section_audio = section_audio_html + updated_md_with_audio

# Save to a new markdown file
final_md_path = "crew-ai-with-section1-audio.md"
with open(final_md_path, "w", encoding="utf-8") as f:
    f.write(updated_md_with_section_audio)

print(f"✅ Final markdown with Section 1 audio saved as '{final_md_path}'")

In [ ]:
# 📌 Cell 8: Re-split original markdown into sections based on raw headings (##, ###)

def split_markdown_by_headings(md_text, heading_level="##"):
    lines = md_text.splitlines()
    sections = []
    current_section = []

    for line in lines:
        if line.strip().startswith(heading_level):
            if current_section:
                sections.append("\n".join(current_section).strip())
                current_section = []
        current_section.append(line)

    if current_section:
        sections.append("\n".join(current_section).strip())

    return sections

# Use updated_md_with_audio which includes Agentic AI intro
raw_md_sections = split_markdown_by_headings(updated_md_with_audio, heading_level="##")

# Preview headings + count
for i, sec in enumerate(raw_md_sections):
    heading = sec.strip().splitlines()[0].strip()
    print(f"🧩 Section {i+1}: {heading}")

print(f"\n✅ Total sections detected: {len(raw_md_sections)}")

In [ ]:
# Check the exact TTS input
for idx in range(2, len(raw_md_sections)):
    section_text = markdown_to_narration_text(raw_md_sections[idx])
    print(f"\n📝 TTS Input for Section {idx}:\n{section_text[:500]}...")

In [ ]:
# 📌 FIXED Cell 10: Generate audio correctly without HTML issues

import time

start_index = 1
total_sections = len(raw_md_sections)

overall_start = time.time()

for idx in range(start_index, total_sections):
    section = raw_md_sections[idx]
    display_index = idx  

    section_start = time.time()

    # ✅ Ensure we only send clean narration text (NO HTML)
    clean_text = markdown_to_narration_text(section)

    # Truncate overly long sections (just in case)
    if len(clean_text) > 4096:
        print(f"⚠️ Truncating Section {display_index} to 4096 characters.")
        clean_text = clean_text[:4096]

    print(f"\n🎙️ [Section {display_index}/{total_sections}] Generating TTS...")

    try:
        response = openai.audio.speech.create(
            model="tts-1",
            voice="echo",
            input=clean_text  # ✅ Ensure no HTML is sent
        )

        filename = f"section_{display_index}.mp3"
        with open(filename, "wb") as f:
            f.write(response.content)

        elapsed = time.time() - section_start
        print(f"✅ Saved: {filename} (⏱ {elapsed:.2f} sec)")

    except Exception as e:
        print(f"❌ Error generating Section {display_index}: {e}")

overall_elapsed = time.time() - overall_start
print(f"\n⏳ All sections processed in {overall_elapsed:.2f} seconds.")

In [ ]:
import os

# List all generated MP3s
audio_files = [f for f in os.listdir() if f.endswith(".mp3")]

# Verify file sizes
for file in audio_files:
    size = os.path.getsize(file)
    print(f"🎵 {file}: {size / 1024:.2f} KB")

In [ ]:
import os

# Delete only the corrupted section_1.mp3 file
if os.path.exists("section_1.mp3"):
    os.remove("section_1.mp3")
    print("🗑 Deleted corrupted section_1.mp3")

# Use OpenAI TTS to regenerate
import openai

# Ensure your API key is set
openai.api_key = os.getenv("OPENAI_API_KEY")

# Get the correct clean text for section_1
section_1_text = markdown_to_narration_text(raw_md_sections[1])  # Section 1 (Index 1)

# Regenerate speech
response = openai.audio.speech.create(
    model="tts-1",
    voice="echo",
    input=section_1_text
)

# Ensure we are saving actual MP3 binary data
if hasattr(response, 'content'):
    with open("section_1.mp3", "wb") as f:
        f.write(response.content)
    print("✅ Successfully regenerated section_1.mp3")
else:
    print("❌ ERROR: OpenAI did not return audio for section_1.")

In [ ]:
# 📌 DEBUG: Verify Cleaned Section 1 (Before Sending to OpenAI)

section_1_text = markdown_to_narration_text(raw_md_sections[1])  # Section 1
print("\n📝 CLEANED TTS Input for Section 1 (First 500 chars):\n", section_1_text[:500])

In [ ]:
# Ensure no unwanted text remains in Section 1
section_1_text = section_1_text.replace("Your browser does not support the audio element.", "").strip()

# Debug again
print("\n📝 FINAL CLEANED TTS Input for Section 1 (First 500 chars):\n", section_1_text[:500])

In [ ]:
# 📌 DEBUG: Check OpenAI's response for section_1 before saving

import openai

# Fetch the clean text again
section_1_text = markdown_to_narration_text(raw_md_sections[1])  # Section 1

print("\n📝 Checking OpenAI TTS Input (First 500 chars):\n", section_1_text[:500])

# Generate speech
response = openai.audio.speech.create(
    model="tts-1",
    voice="echo",
    input=section_1_text
)

# 🛠 DEBUG: Check if response contains audio or text
print("\n🔍 OpenAI API Response Type:", type(response))
print("\n🔍 OpenAI API Response Content (First 500 chars):\n", response.content[:500])

# Check response headers to confirm it's MP3
print("\n🔍 OpenAI API Response Headers:", response.headers if hasattr(response, 'headers') else "No headers available")

# Play the response (if valid audio)
import IPython.display as ipd
ipd.Audio(response.content)

In [ ]:
# Ensure no unwanted text remains in Section 1
section_1_text = section_1_text.replace("Your browser does not support the audio element.", "").strip()

# Debug again
print("\n📝 FINAL CLEANED TTS Input for Section 1 (First 500 chars):\n", section_1_text[:500])

# 📌 Generate Clean TTS for Section 1

response = openai.audio.speech.create(
    model="tts-1",
    voice="echo",
    input=section_1_text
)

# Save new MP3
if hasattr(response, 'content') and isinstance(response.content, bytes):
    with open("section_1.mp3", "wb") as f:
        f.write(response.content)
    print("✅ Successfully regenerated CLEAN section_1.mp3")
else:
    print("❌ OpenAI did NOT return valid speech. Check the API response.")

In [ ]:
import os
import IPython.display as ipd

# Select one of the MP3 files
sample_mp3 = "section_1.mp3"  # Change to another file if needed

# Play audio to verify if the issue exists
print(f"🔍 Playing: {sample_mp3}")
ipd.Audio(sample_mp3)

In [ ]:
# 📌 Cell 11: Embed all section audio players and save as 'podcast-on-agentic-crew-ai.md'

# Start with markdown containing Agentic AI intro
final_md = updated_md

# Audio embed helper
def audio_html(filename):
    return (
        f'\n<audio controls>\n'
        f'  <source src="./{filename}" type="audio/mpeg">\n'
        f'  Your browser does not support the audio element.\n'
        f'</audio>\n'
    )

# Inject audio after each heading based on section titles
for idx in range(2, len(raw_md_sections)):
    section_heading_line = raw_md_sections[idx].splitlines()[0].strip()
    heading_text = re.sub(r'^#+\s*', '', section_heading_line)
    pattern = re.compile(rf"^(#+\s*{re.escape(heading_text)})", re.MULTILINE)
    final_md = pattern.sub(rf"\1\n{audio_html(f'section_{idx}.mp3')}", final_md)

# Inject your intro after "Introduction to Agentic AI"
final_md = final_md.replace(
    "## Introduction to Agentic AI",
    "## Introduction to Agentic AI\n" + audio_html("intro_prabu.mp3"),
    1
)

# Save to renamed markdown file
final_output_path = "podcast-on-agentic-crew-ai.md"
with open(final_output_path, "w", encoding="utf-8") as f:
    f.write(final_md)

print(f"✅ Final podcast markdown saved as '{final_output_path}'")

In [ ]:
# 📌 Cell 12 (fixed): Generate Podcast Q&A using new OpenAI Python SDK format

import os
from openai import OpenAI

# Initialize OpenAI client using API key from environment
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Combine relevant markdown sections for the podcast
sections_to_include = [1, 2, 5, 6, 7, 10, 11]  # Indexes from raw_md_sections
combined_text = "\n\n".join(markdown_to_narration_text(raw_md_sections[i]) for i in sections_to_include)

# Prompt definition
messages = [
    {
        "role": "system",
        "content": "You are a podcast producer converting technical documentation into a natural, engaging interview script between a host and an expert named Prabu Arumugam."
    },
    {
        "role": "user",
        "content": f"""
Turn the following documentation into a podcast-style Q&A. Include:
- An intro from the host
- A greeting and personal intro from Prabu (as already recorded)
- Conversational tone (not robotic)
- Questions from host followed by expert answers

Keep responses concise and engaging. Here is the content:

{combined_text}
        """
    }
]

# Call OpenAI API
response = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=messages,
    temperature=0.7
)

# Extract the script
podcast_script = response.choices[0].message.content.strip()

# Print preview (first 2000 characters)
print(podcast_script[:2000])

In [ ]:
# 📌 Cell 13 (fixed): Safely split podcast script into speaker segments

segments = []
lines = podcast_script.strip().splitlines()

current_speaker = None
current_text = []

for line in lines:
    line = line.strip()
    if line.startswith("Host:"):
        if current_text:
            segments.append({
                "speaker": current_speaker or "narration",
                "text": " ".join(current_text)
            })
            current_text = []
        current_speaker = "host"
        current_text.append(line.replace("Host:", "").strip())
    elif line.startswith("Prabu:"):
        if current_text:
            segments.append({
                "speaker": current_speaker or "narration",
                "text": " ".join(current_text)
            })
            current_text = []
        current_speaker = "prabu"
        current_text.append(line.replace("Prabu:", "").strip())
    else:
        current_text.append(line)

# Append last collected segment
if current_text:
    segments.append({
        "speaker": current_speaker or "narration",
        "text": " ".join(current_text)
    })

# Display preview
for i, seg in enumerate(segments):
    speaker = seg["speaker"]
    preview = seg["text"][:80].strip()
    print(f"[{i+1}] 🎙️ {speaker.title()} says: {preview}...")

print(f"\n✅ Total segments: {len(segments)}")

In [ ]:
from openai import OpenAI
import os

# Initialize OpenAI client
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Define voice mapping
voice_map = {
    "narration": "echo",  # Neutral voice for narration
    "host": "nova",       # Friendly voice for host
    "prabu": "onyx"       # Authoritative, deeper voice for Prabu
}

# Ensure podcast_script is properly populated with the podcast text
podcast_script = """Host: Hello and welcome to today's podcast!
Prabu: Thanks for having me, it's a pleasure to be here.
Host: Today we will talk about Agentic AI.
Prabu: Absolutely, let's dive right into it.
Host: Can you explain what Agentic AI is?
Prabu: Of course, Agentic AI is the next evolution in AI technology.
Host: That sounds interesting, tell us more.
Prabu: Sure, it works by coordinating specialized agents to work together autonomously."""

# Split the podcast script into segments based on speakers (Host and Prabu)
segments = []
lines = podcast_script.strip().splitlines()

current_speaker = None
current_text = []

for line in lines:
    line = line.strip()
    if line.startswith("Host:"):
        if current_text:
            segments.append({
                "speaker": current_speaker or "narration",
                "text": " ".join(current_text)
            })
            current_text = []
        current_speaker = "host"
        current_text.append(line.replace("Host:", "").strip())
    elif line.startswith("Prabu:"):
        if current_text:
            segments.append({
                "speaker": current_speaker or "narration",
                "text": " ".join(current_text)
            })
            current_text = []
        current_speaker = "prabu"
        current_text.append(line.replace("Prabu:", "").strip())
    else:
        current_text.append(line)

# Append the last collected segment
if current_text:
    segments.append({
        "speaker": current_speaker or "narration",
        "text": " ".join(current_text)
    })

# Display preview of segments
for i, seg in enumerate(segments):
    speaker = seg["speaker"]
    preview = seg["text"][:80].strip()
    print(f"[{i+1}] 🎙️ {speaker.title()} says: {preview}...")

print(f"\n✅ Total segments: {len(segments)}")

# Now generate TTS MP3 for each segment and save them
output_dir = "tts_segments"
os.makedirs(output_dir, exist_ok=True)

# Loop through segments and generate TTS for each
for i, seg in enumerate(segments, 1):
    speaker = seg["speaker"]
    text = seg["text"].strip()  # Clean up the text to avoid extra spaces or empty segments
    
    if not text:
        print(f"❌ Skipping segment {i}: No text available.")
        continue  # Skip empty segments

    voice = voice_map.get(speaker, "echo")  # Default to "echo" if no voice found
    filename = f"podcast_segment_{i:03d}_{speaker}.mp3"
    filepath = os.path.join(output_dir, filename)

    print(f"🎤 [{i}/{len(segments)}] Generating {filename} with voice '{voice}'...")

    try:
        # Generate the TTS response from OpenAI API
        response = client.audio.speech.create(
            model="tts-1-hd",  # Use the TTS model
            voice=voice,       # Select the voice based on the speaker
            input=text         # Pass the text for conversion to speech
        )

        # Save the generated audio to the file
        if 'data' in response:
            # Ensure the response data is being written to the file as binary content
            with open(filepath, "wb") as f:
                f.write(response['data'])
            print(f"✅ {filename} saved successfully!")
        else:
            print(f"❌ No audio content in the response for segment {i}")

    except Exception as e:
        print(f"❌ Error generating {filename}: {e}")

print("\n✅ All segments processed. Audio files saved in:", output_dir)